In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,accuracy_score,roc_curve,roc_auc_score,confusion_matrix
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
%matplotlib inline

In [2]:
data = pd.read_csv("../Training_Batch_Files/creditCardFraud_28011961_120210.csv")
data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [3]:
x = data.drop(columns="default.payment.next.month")
x.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000.0,2,2,1,24,2,2,-1,-1,-2,...,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0
1,120000.0,2,2,2,26,-1,2,0,0,0,...,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0
2,90000.0,2,2,2,34,0,0,0,0,0,...,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0
3,50000.0,2,2,1,37,0,0,0,0,0,...,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0
4,50000.0,1,2,1,57,-1,0,-1,0,0,...,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0


In [4]:
y = data['default.payment.next.month']
y.head()

0    1
1    1
2    0
3    0
4    0
Name: default.payment.next.month, dtype: int64

In [5]:
# spliting data into train and test
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=100)

In [6]:
x_train.shape

(24000, 23)

In [7]:
y_train.shape

(24000,)

In [8]:
x_test.shape

(6000, 23)

In [9]:
y_test.shape

(6000,)

In [10]:
# Initializing standard scaler object
train_scaler = StandardScaler()
test_scaler = StandardScaler()

In [11]:
scaled_train_data=train_scaler.fit_transform(x_train)
scaled_test_data=test_scaler.fit_transform(x_test)

In [ ]:
pickle.dump(self.scaled_train_data, open('models/scaler/scaled_data_train.pkl', 'wb'))

In [12]:
scaled_train_df=pd.DataFrame(data=scaled_train_data, columns=x_train.columns, index=x_train.index)

In [13]:
scaled_test_df=pd.DataFrame(data=scaled_test_data, columns=x_test.columns, index=x_test.index)

In [14]:
scaled_train_df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
795,-0.521218,0.809498,0.184920,-1.055753,-0.595947,0.018726,0.116622,0.143771,0.197220,0.243774,...,0.400949,0.515036,0.620763,0.690497,-0.168409,-0.133410,0.008799,-0.082417,-0.088686,-0.115188
23652,-1.136449,0.809498,-1.081295,-1.055753,0.160668,0.906908,-1.553386,-0.692918,1.917785,2.019101,...,-0.519449,-0.509328,-0.665178,-0.654181,-0.239433,0.211102,-0.291994,-0.305623,-0.316181,-0.011962
9038,-0.059796,0.809498,0.184920,-1.055753,-0.487859,0.018726,0.116622,0.143771,0.197220,0.243774,...,0.424721,0.563247,0.699203,0.770936,-0.040856,0.169594,-0.013482,0.013243,-0.056187,-0.071724
11588,0.863049,-1.235334,0.184920,0.856555,-0.595947,-0.869457,-0.718382,-0.692918,-0.663063,-0.643889,...,-0.666387,-0.660174,-0.650484,-0.639231,-0.235084,-0.218459,-0.255732,-0.264107,-0.273867,-0.289042
7422,-0.828833,0.809498,-1.081295,0.856555,-1.136386,0.018726,0.116622,0.143771,1.917785,2.019101,...,-0.287430,-0.263575,-0.198691,-0.170199,-0.248767,-0.134323,-0.291994,-0.158307,-0.239937,-0.289042


## Gaussian Naive-Bayes

In [15]:
gnb=GaussianNB()

In [17]:
gnb.fit(scaled_train_df,y_train)

GaussianNB()

In [19]:
gnbPred_y = gnb.predict(scaled_test_df)

In [20]:
acGNB = accuracy_score(y_test,gnbPred_y)
acGNB

0.752

In [21]:
param_grid = {"var_smoothing": [1e-9,0.1, 0.001, 0.5,0.05,0.01,1e-8,1e-7,1e-6,1e-10,1e-11]}
#Creating an object of the Grid Search class
grid = GridSearchCV(estimator=gnb, param_grid=param_grid, cv=5,  verbose=3)

In [22]:
grid.fit(scaled_train_data,y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
[CV 1/5] END ...............var_smoothing=1e-09;, score=0.733 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-09;, score=0.725 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-09;, score=0.750 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-09;, score=0.709 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-09;, score=0.743 total time=   0.0s
[CV 1/5] END .................var_smoothing=0.1;, score=0.765 total time=   0.0s
[CV 2/5] END .................var_smoothing=0.1;, score=0.766 total time=   0.0s
[CV 3/5] END .................var_smoothing=0.1;, score=0.770 total time=   0.0s
[CV 4/5] END .................var_smoothing=0.1;, score=0.759 total time=   0.0s
[CV 5/5] END .................var_smoothing=0.1;, score=0.774 total time=   0.0s
[CV 1/5] END ...............var_smoothing=0.001;, score=0.734 total time=   0.0s
[CV 2/5] END ...............var_smoothing=0.001;

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-09, 0.1, 0.001, 0.5, 0.05, 0.01,
                                           1e-08, 1e-07, 1e-06, 1e-10, 1e-11]},
             verbose=3)

In [23]:
grid.best_estimator_

GaussianNB(var_smoothing=0.5)

In [24]:
# retrianing after hyperparamter tuning
gnb_new=GaussianNB(var_smoothing=0.5)

In [25]:
gnb_new.fit(scaled_train_df,y_train)

GaussianNB(var_smoothing=0.5)

In [26]:
gnbPred_yNew = gnb_new.predict(scaled_test_df)

In [27]:
acGNB_New = accuracy_score(y_test,gnbPred_yNew)

In [28]:
acGNB_New

0.7996666666666666

## XGBoost

In [37]:
xb = xgb.XGBClassifier(objective='binary:logistic')

In [38]:
xb.fit(scaled_train_df,y_train)

/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:44:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [39]:
xgb_pred = xb.predict(scaled_test_df)
xgb_pred

array([0, 1, 0, ..., 0, 0, 0])

In [40]:
ac_xgb = accuracy_score(y_test,xgb_pred)
ac_xgb

0.8091666666666667

In [41]:
param_grid_xgboost = {
    "n_estimators": [50,100, 130],
    "max_depth": range(3, 11, 1),
    "random_state":[0,50,100]
}

In [42]:
grid = GridSearchCV(xb,param_grid_xgboost,verbose=3,n_jobs=-1)

In [43]:
grid.fit(scaled_train_data,y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/swapnil/anaconda3/envs/project/lib/python3

/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/swapnil/anaconda3/envs/project/lib/python3

/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/swapnil/anaconda3/envs/project/lib/python3

/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/swapnil/anaconda3/envs/project/lib/python3

...

[22:01:45] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1,
                                     enable_categorical=False, gamma=0,
                                     gpu_id=-1, importance_type=None,
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_estimators=100, n_jobs=8,
                                     num_parallel_tree=1, predictor='auto',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
                              


[18:53:08] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5] END max_depth=7, n_estimators=100, random_state=0;, score=0.815 total time= 6.4min
[18:59:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 4/5] END max_depth=7, n_estimators=100, random_state=50;, score=0.807 total time= 7.1min
[19:06:39] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5] END max_depth=7, n_estimators=130, random_state=0


[18:52:16] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5] END max_depth=7, n_estimators=50, random_state=100;, score=0.813 total time= 3.4min
[18:55:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5] END max_depth=7, n_estimators=100, random_state=50;, score=0.815 total time= 6.1min
[19:01:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 4/5] END max_depth=7, n_estimators=100, random_state=


[18:56:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5] END max_depth=7, n_estimators=100, random_state=50;, score=0.811 total time= 6.2min
[19:02:11] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 5/5] END max_depth=7, n_estimators=100, random_state=100;, score=0.809 total time= 7.3min
[19:09:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 3/5] END max_depth=7, n_estimators=130, random_state

In [44]:
grid.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=50, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [59]:
xgb_new = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1,
              gamma=0, gpu_id=-1,learning_rate=0.300000012,
              max_delta_step=0, max_depth=3, min_child_weight=1, missing=1,
              n_estimators=50, n_jobs=8,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', verbosity=1)

In [61]:
xgb_new.fit(scaled_train_df,y_train)

[22:36:00] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=3, min_child_weight=1, missing=1,
              monotone_constraints='()', n_estimators=50, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=1)

In [62]:
xgb_new_pred = xgb_new.predict(scaled_test_df)
xgb_new_pred

array([0, 0, 0, ..., 0, 0, 0])

In [63]:
ac_xgb_new = accuracy_score(y_test,xgb_new_pred)
ac_xgb_new

0.823